# Gemini File Search Tool

This notebook demonstrates how to use the Google GenAI SDK to create a File Search Store, upload PDF documents, and perform grounded queries using Gemini's built-in file search capabilities.

In [11]:
%pip install python-dotenv google-genai


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from google import genai
from google.genai import types
import time
from IPython.display import Markdown, HTML, display
from dotenv import load_dotenv

# Load environment variables (API Key)
load_dotenv()
client = genai.Client()

## Visualization Helpers

We'll define a function to render our grounding sources in a clean, interactive way.

In [18]:
def display_grounding_metadata(response):
    """Renders the AI response and sources with premium formatting."""
    # 1. Display the Main Response
    display("### 🤖 Gemini Response")
    display(response.text)
    display(HTML("<hr style='border: 0.5px solid #eee; margin: 20px 0;'>"))

    # 2. Display Sources
    display(Markdown("#### 📚 Sources Used"))
    
    metadata = response.candidates[0].grounding_metadata
    if not metadata or not metadata.grounding_chunks:
        print("No specific grounding sources found.")
        return

    cards_html = "<div style='display: flex; flex-wrap: wrap; gap: 15px;'>"
    
    for i, chunk in enumerate(metadata.grounding_chunks, 1):
        title = chunk.retrieved_context.title or "Untitled Source"
        text = chunk.retrieved_context.text[:200] + "..." if chunk.retrieved_context.text else "No snippet available"
        
        cards_html += f"""
        <div style='
            flex: 1; 
            min-width: 250px; 
            padding: 15px; 
            border-radius: 8px; 
            background: #fdfdfd; 
            border: 1px solid #e1e4e8; 
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        '>
            <div style='font-weight: 600; color: #1a73e8; margin-bottom: 8px;'>[{i}] {title}</div>
            <div style='font-size: 0.9em; color: #5f6368; line-height: 1.4;'>{text}</div>
        </div>
        """
    
    cards_html += "</div>"
    display(HTML(cards_html))

## 1. Create File Search Store

First, we create a specialized store to hold our drug labels.

In [15]:
file_search_store = client.file_search_stores.create(
    config={"display_name": "hb-cozummerkezi"}
)
print(f"Created store: {file_search_store.name}")

Created store: fileSearchStores/hbcozummerkezi-y0xseba6i1oz


## 2. Upload and Index PDFs

We upload the PDF files and wait for the background indexing process to complete.

In [ ]:
pdf_files = ["data/hb_iade.pdf", "data/hb_kargo.pdf", "data/hb_odeme.pdf"]

for pdf_file in pdf_files:
    operation = client.file_search_stores.upload_to_file_search_store(
        file=pdf_file,
        file_search_store_name=file_search_store.name,
        config={"display_name": pdf_file.replace(".pdf", "")},
    )

    # Wait for indexing to complete
    while not operation.done:
        print(f"Indexing {pdf_file}...", end="\r")
        time.sleep(3)
        operation = client.operations.get(operation)

    print(f"- {pdf_file} indexed ")

- data/hb_iade.pdf indexed .
- data/hb_kargo.pdf indexed .
- data/hb_odeme.pdf indexed .


## 3. Grounded Retrieval - Query 1

**Question**: Hangi kredi kartları ile ödeme yapabilirim?

In [19]:
query1 = "Hangi kredi kartları ile ödeme yapabilirim?"

response1 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=query1,
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    ),
)

display_grounding_metadata(response1)

'### 🤖 Gemini Response'

'Hepsiburada üzerinden alışveriş yaparken Visa, Mastercard ve Troy kartları ile ödeme yapabilirsiniz. Kart sahibinin bankası tarafından uygulanan güvenlik kontrolleri geçerli olup, bazı işlemler için 3D Secure doğrulaması zorunlu olabilir. Eğer ödeme başarısız olursa sipariş otomatik olarak iptal edilir.\n\nÖdeme yöntemleri arasında kredi kartının yanı sıra banka kartı, Hepsiburada Cüzdan, hediye çeki ve promosyon kodları da bulunmaktadır. Ayrıca belirli ürün ve satıcılarda kapıda ödeme seçeneği de mevcuttur. Kapıda ödeme seçeneği nakit veya kredi kartı ile yapılabilir ve ek hizmet bedeli yansıtılabilir. Yüksek tutarlı veya büyük hacimli ürünlerde kapıda ödeme seçeneği sunulmayabilir.\n\nTaksit imkanları banka ve kampanyaya göre 2 ila 12 taksit arasında değişebilir, ancak elektronik ürünlerde taksit sayısı yasal düzenlemeler gereği sınırlandırılabilir. Bazı ürünlerde yalnızca tek çekim yapılabilmektedir. Taksit seçenekleri ödeme ekranında gösterilir.'

#### 📚 Sources Used

## 4. Grounded Retrieval - Query 2

**Question**: Kargoya verilmiş bir ürün için iade sürecini nasıl başlatabilirim?

In [ ]:
query2 = "Kargoya verilmiş bir ürün için iade sürecini nasıl başlatabilirim?"

response2 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=query2,
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    ),
)

display_grounding_metadata(response2)

## 5. Grounded Retrieval - Query 3

**Question**: Kargoya verilen için ödeme yöntemini değiştirmek mümkün mü?

In [ ]:
query3 = "Kargoya verilen için ödeme yöntemini değiştirmek mümkün mü"

response3 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=query3,
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    ),
)

display_grounding_metadata(response3)